In [53]:
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from keras.regularizers import l2
MODEL_PATH = './model/'
DATA_PATH = './data/'

In [79]:
target_dataset = pd.read_csv('../MIA/datasets/adultODS10K.csv', na_values=["?"])
shadow_dataset = pd.read_csv('../MIA/datasets/adultODS10K_to_25K.csv', na_values=["?"])
attack_test_nonmembers = pd.read_csv('data/adultODS25K_to_28K.csv', na_values=["?"])
attack_test_members = pd.read_csv('data/adultODS1_to_3K.csv', na_values=["?"])

In [55]:
def read_data(data_name):
    with np.load(DATA_PATH + data_name) as f:
        train_x, train_y, test_x, test_y = [f['arr_%d' % i] for i in range(len(f.files))]
    return train_x, train_y, test_x, test_y

In [80]:
def transform_data(dataset):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    y=to_categorical(y)
    
    return x, y

In [84]:
def load_target_data(dataset, test_ratio):
    x, y = transform_data(dataset)
    
    trainX,testX, trainY, testY = train_test_split(x, y, test_size=test_ratio, random_state=0, stratify=y)
    return (trainX, trainY), (testX, testY), dim

In [85]:
def build_simple_mlp(n_class,pix,d):

    model = Sequential()
    model.add(Dense(256, input_dim=pix))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    model.add(Dense(256, kernel_regularizer=l2(0.01)))
    model.add(Activation("relu"))
    #model.add(Dropout(0.01))
    
    
    #model.add(Dense(248))
    #model.add(Activation("relu"))
    #model.add(Dropout(0.01))

    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.01))
    
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    act_layer=3
    
    return model, act_layer

In [86]:
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
VERBOSE = 0
test_ratio = 0.3

(target_trainX, target_trainY), (target_testX, target_testY), dim = load_target_data(target_dataset, test_ratio)
target_model,_ = build_simple_mlp (n_class,dim,channel)
#get_trained_keras_models(model, (target_trainX, target_trainY), (target_testX, target_testY), num_models=1)
history = target_model.fit(target_trainX, target_trainY, epochs=EPS, batch_size=32, verbose=VERBOSE)
score = target_model.evaluate(target_testX, target_testY, verbose=VERBOSE)
print('\n', 'Model test accuracy:', score[1])


 Model test accuracy: 0.8270000219345093


In [72]:
def load_shadow_data(dataset, n_shadow, shadow_size, test_ratio):
    x = dataset.iloc[:,0:dataset.shape[1]-1] # seperate the feature column from class label
    y = dataset.iloc[:,-1] # label column

    dim=x.shape[1] # number of feature columns

    #num of classes
    num_classes=2

    for j in range(0,dim):
        if x.iloc[:,j].dtypes == object:   # transform categorical variables
            x.iloc[:,j] = x.iloc[:,j].astype('category') # change datatype to categorical
            x.iloc[:,j] = x.iloc[:,j].cat.codes # change from category name to category number
        else:  #transform numrical variables to standard scaler form 
            sc = StandardScaler()  
            val=np.array(x.iloc[:,j]).reshape(-1,1)
            std_data = sc.fit_transform(val)
            std_data = pd.DataFrame(std_data)
            x.iloc[:,j]=std_data

    y = y.astype('category') # change label to categorical
    y = y.cat.codes # change from category name to number

    x=np.array(x)
    y=np.array(y)
    
    y=to_categorical(y)
    
    shadow_indices = np.arange(len(dataset))
    
   
    for i in range(n_shadow):
        shadow_i_indices = np.random.choice(shadow_indices, shadow_size, replace=False)
        shadow_i_x, shadow_i_y = x[shadow_i_indices], y[shadow_i_indices]
        trainX,testX, trainY, testY = train_test_split(shadow_i_x, shadow_i_y, test_size=test_ratio, random_state=0, stratify=shadow_i_y)
        #print('shadow_i_trainX = ', trainX, 'shadow_i_trainY = ', trainY)
        
        np.savez(DATA_PATH + 'shadow_adult{}_data.npz'.format(i), trainX, trainY, testX, testY)

In [73]:
n_shadow_models = 10
shadow_data_size = 10000
test_ratio = 0.3

load_shadow_data(shadow_dataset, 10, 10000, 0.3)

In [75]:
def train_shadow_models(n_shadow, n_class, dim, channel):
    full_sm_train_pred=[]
    full_sm_train_class=[]
    
    full_sm_test_pred=[]
    full_sm_test_class=[]
    
    full_clz_train=[]
    full_clz_test=[]
    
    members=[]
    nonmembers=[]
    

    for j in range(n_shadow):
        
        print("Shadow Model ", j)
        
        print('Training shadow model {}'.format(j))
        data = read_data('shadow_adult{}_data.npz'.format(j))
        x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test = data
        #print('x_shadow trian\n', x_shadow_train,'\n y_shadow trian\n', y_shadow_train, '\n x_shadow test\n', x_shadow_test, '\n y_shadow test\n', y_shadow_test)

                

        model,_ = build_simple_mlp(n_class,dim,channel)
            
        # fit model
        history = model.fit(x_shadow_train, y_shadow_train, epochs=EPS, batch_size=32, validation_data=(x_shadow_test, y_shadow_test), verbose=0)
    
        # evaluate model
        _, train_acc = model.evaluate(x_shadow_train, y_shadow_train, verbose=0)
        _, test_acc = model.evaluate(x_shadow_test, y_shadow_test, verbose=0)
        print("Shadow Train acc : ", (train_acc * 100.0),"Shadow Test acc : ", (test_acc * 100.0))

    
        #train SM
        sm_train_pred=model.predict(x_shadow_train, batch_size=32)
        sm_train_class=np.argmax(y_shadow_train,axis=1)
    
    
        #test SM
        sm_test_pred=model.predict(x_shadow_test, batch_size=32)
        sm_test_class=np.argmax(y_shadow_test,axis=1)
        
     
        full_sm_train_pred.append(sm_train_pred)        
        full_sm_train_class.append(sm_train_class)
        members.append(np.ones(len(sm_train_pred)))
        
        full_sm_test_pred.append(sm_test_pred)        
        full_sm_test_class.append(sm_test_class) 
        nonmembers.append(np.zeros(len(sm_test_pred)))

#     print("\n train predic", full_sm_train_pred)
#     print("\n trian class", full_sm_train_class)
#     print("\n members", members)
#     print("\n nonmembers", nonmembers)
    
#     print("\n train class type ", type(full_sm_train_class))

    
    full_sm_train_pred = np.vstack(full_sm_train_pred)
    full_sm_train_class = [item for sublist in full_sm_train_class for item in sublist]
    members = [item for sublist in members for item in sublist]
    
    full_sm_test_pred = np.vstack(full_sm_test_pred)
    full_sm_test_class = [item for sublist in full_sm_test_class for item in sublist]
    
    nonmembers = [item for sublist in nonmembers for item in sublist]
    

    
    shadow_train_performance=(full_sm_train_pred, np.array(full_sm_train_class))
    shadow_test_performance=(full_sm_test_pred, np.array(full_sm_test_class))


    ###atack data preparation
    attack_x = (full_sm_train_pred,full_sm_test_pred)
    #attack_x = np.vstack(attack_x)
    
    attack_y = (np.array(members).astype('int32'),np.array(nonmembers).astype('int32'))
    #attack_y = np.concatenate(attack_y)
    #attack_y = attack_y.astype('int32')
    
    
    classes = (np.array(full_sm_train_class),np.array(full_sm_test_class))
    #classes = np.array([item for sublist in classes for item in sublist])


    attack_dataset = (attack_x,attack_y,classes)
    
    #print('attack dataset\n', attack_dataset)
    
    #print(shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model)

            
    return  shadow_train_performance, shadow_test_performance, attack_dataset, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, model


In [76]:
per_class_sample=5000
channel=0   
EPS=200
act_layer=3
n_class = 2
VERBOSE = 0
test_ratio = 0.3

n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init = train_shadow_models(n_shadow_models, n_class, dim, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  89.07142877578735 Shadow Test acc :  81.80000185966492
94/94 [==============================] - 0s 1ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  88.34285736083984 Shadow Test acc :  81.76666498184204
94/94 [==============================] - 0s 1ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  87.91428804397583 Shadow Test acc :  82.70000219345093
94/94 [==============================] - 0s 1ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  88.44285607337952 Shadow Test acc :  82.4666678905487
94/94 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  88.38571310043335 Shadow Test acc :  80.96666932106018
94/94 [==============================] - 0s 1ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  86.88571453094482 Shadow Test acc :  82.16666579246521
94/94 [==============================] - 0s 2m

In [78]:
np.savez(DATA_PATH + 'attack_adult_data.npz', n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init)

INFO:tensorflow:Assets written to: ram://dc3f0c4a-c7dd-4766-bcb8-6bacd1689653/assets


In [ ]:
def define_attack_model(n_class):
    model = Sequential()
    
    model.add(Dense(1))
    model.add(Activation("relu"))
    
    model.add(Dense(1))
    model.add(Activation("relu"))

    model.add(Dense(n_class, activation='softmax'))

    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def load_attack_test_data():
    
    

In [ ]:
def train_attack_model(attack_data, check_membership, n_hidden=50, learning_rate=0.01, batch_size=200, epochs=50, model='nn', l2_ratio=1e-7):

    x, y,  classes = attack_data

    train_x = x[0]
    train_y = y[0]
    test_x = x[1]
    test_y = y[1]
    train_classes = classes[0]
    test_classes = classes[1]
    
    
    checkmem_prediction_vals, checkmem_membership_status, checkmem_class_status = check_membership
    
    checkmem_prediction_vals=np.vstack(checkmem_prediction_vals)
    checkmem_membership_status=np.array([item for sublist in checkmem_membership_status for item in sublist])
    checkmem_class_status=np.array([item for sublist in checkmem_class_status for item in sublist])
    
    train_indices = np.arange(len(train_x))
    test_indices = np.arange(len(test_x))
    unique_classes = np.unique(train_classes)


    predicted_membership, target_membership = [], []
    for c in unique_classes:
        print("Class : ", c)
        c_train_indices = train_indices[train_classes == c]
        c_train_x, c_train_y = train_x[c_train_indices], train_y[c_train_indices]
        c_test_indices = test_indices[test_classes == c]
        c_test_x, c_test_y = test_x[c_test_indices], test_y[c_test_indices]
        c_dataset = (c_train_x, c_train_y, c_test_x, c_test_y)        
        
        full_cx_data=(c_train_x,c_test_x)
        full_cx_data = np.vstack(full_cx_data)

        full_cy_data=(c_train_y,c_test_y)
        full_cy_data = np.array([item for sublist in full_cy_data for item in sublist])

                
        
        classifier = define_attack_model(2)
        history = classifier.fit(full_cx_data, full_cy_data, epochs=EPS, batch_size=32, verbose=0)

        #get predictions on real train and test data
        c_indices = np.where(checkmem_class_status==c)
        c_pred_y = classifier.predict_classes(checkmem_prediction_vals[c_indices])
        c_target_y = checkmem_membership_status[c_indices]
        
       
        target_membership.append(c_target_y)
        predicted_membership.append(c_pred_y)

    target_membership=np.array([item for sublist in target_membership for item in sublist])
    predicted_membership=np.array([item for sublist in predicted_membership for item in sublist])


        
    tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (target_membership,predicted_membership)   
    return tp, fp, fn, tn, precision, advj, acc, recall